In [1]:
import pandas as pd
import os
import cv2

In [4]:
folder_train = 'data/img/train/'
folder_valid = 'data/img/valid/'

makesenseai_train = folder_train+'train.csv'
makesenseai_valid = folder_valid+'valid.csv'



In [5]:
def read_makesense_ai_output(
    makesenseai_path
):
#     folder = 'img/train/'
    makesenseai_df = pd.read_csv(
        #folder+'labels.csv',
        makesenseai_path,
        header=None,
        names=[
            'class', 
            'x','y','w','h', 
            'file_name', 
            'image_width','image_height'
        ]
    )
    return makesenseai_df
makesenseai_train_df = read_makesense_ai_output(makesenseai_train)
makesenseai_train_df

,class,x,y,w,h,file_name,image_width,image_height
0,Person,102,217,26,24,0.png,496,360
1,Person,210,212,20,26,0.png,496,360
2,Person,84,192,44,49,1.png,496,360
3,Person,209,214,22,25,1.png,496,360
4,Person,209,214,20,24,10.png,496,360
5,Person,307,223,38,36,10.png,496,360
6,Person,211,214,18,23,11.png,496,360
7,Person,307,225,41,35,11.png,496,360
8,Person,209,212,27,27,12.png,496,360
9,Person,304,218,41,42,12.png,496,360


In [7]:
group = makesenseai_train_df.groupby(['file_name'])
group

In [12]:
for g, content in group:
    print(g)
    display(content)

0.png


,class,x,y,w,h,file_name,image_width,image_height
0,Person,102,217,26,24,0.png,496,360
1,Person,210,212,20,26,0.png,496,360


1.png


,class,x,y,w,h,file_name,image_width,image_height
2,Person,84,192,44,49,1.png,496,360
3,Person,209,214,22,25,1.png,496,360


10.png


...

,class,x,y,w,h,file_name,image_width,image_height
20,Person,214,185,45,65,7.png,496,360


8.png


,class,x,y,w,h,file_name,image_width,image_height
21,Person,211,216,23,23,8.png,496,360
22,Person,272,222,37,34,8.png,496,360


9.png


,class,x,y,w,h,file_name,image_width,image_height
23,Person,209,214,20,24,9.png,496,360
24,Person,287,205,46,43,9.png,496,360


In [50]:
def get_images_names_range(makesenseai_df, start=0):
    images_count = makesenseai_df['file_name'].unique().shape[0]
    # files = range(args.start, args.start-1+length) # -e.g.- ['0.png', '1.png', ...]
    images_names_range = range(start, start+images_count) # -e.g.- ['0.png', '1.png', ...]
    return images_names_range

from detectron2.structures import BoxMode
def makesenseai_to_COCO(makesenseai_df, folder, start=0):
    images_names_range = get_images_names_range(
        makesenseai_df, start=start
    )
    COCO = []
    for i in images_names_range:
        current_image = {}
        current_image['file_name'] = str(i)+'.png'
        current_image['image_id'] = int(i)
        img = cv2.imread(
            os.path.join(folder, current_image['file_name'])
        )
        height, width, channels = img.shape
        current_image['height'] = int(height)
        current_image['width'] = int(width)
        
        
        current_image_rows = makesenseai_df[
            makesenseai_df["file_name"] \
                == current_image['file_name']
        ]
#         if current_image_rows.shape[0] > 0:
        current_image['annotations'] = []
        
        for i in range(current_image_rows.shape[0]):
            label = {}
            row = current_image_rows.iloc[i]
            label['bbox'] = int(row['x']), int(row['y']), \
                            int(row['w']), int(row['h'])
            label['bbox_mode'] = BoxMode.XYWH_ABS
#             label['segmentation'] = [[916.5,515.5,588.5,173.5,]]
            label['category_id'] = 0
            current_image['annotations'].append(label)
        COCO.append(current_image)
    return COCO
COCO_train = \
    makesenseai_to_COCO(
        makesenseai_train_df, folder_train, start=0
    )
COCO_train

[{'file_name': '0.png',
  'image_id': 0,
  'height': 360,
  'width': 496,
  'annotations': [{'bbox': (102, 217, 26, 24),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0},
   {'bbox': (210, 212, 20, 26),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0}]},
 {'file_name': '1.png',
  'image_id': 1,
  'height': 360,
  'width': 496,
  'annotations': [{'bbox': (84, 192, 44, 49),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0},
   {'bbox': (209, 214, 22, 25),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0}]},
 {'file_name': '2.png',
  'image_id': 2,
  'height': 360,
  'width': 496,
  'annotations': [{'bbox': (78, 178, 63, 64),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0},
   {'bbox': (210, 217, 20, 21),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 0}]},
 {'file_name': '3.png',
  'image_id': 3,
  'height': 360,
  'width': 496,
  'annotations': [{'bbox': (209, 213, 20, 26),
    'bbox_mode': <BoxMode.XYWH_ABS: 1>

In [51]:
import json
def save_COCO_file(path_to_save, COCO):
    with open(path_to_save, "w") as file_handle:
        json.dump(
            COCO, file_handle, 
            indent=4, sort_keys=False
        )
save_COCO_file(folder_train+'train.json', COCO_train)